In [1]:
%matplotlib notebook
from numpy import *
from qutip import *
from typing import Tuple

from basics import *
from hybrid_matrix_elements import *
from atomic_physics import *

In [2]:
rme = q_rme_hf_h(fp=6) # <5D5/2,6'||Q||6S1/2,4> Quadrupole RME
k = SphericalVector([1,0,0],spherical=False)
q = SphericalVector([1,0,0],spherical=True)



TypeError: q_rme_hf_h() missing 1 required positional argument: 'q_rad'

In [ ]:
basis = 